In [1]:
from __future__ import division, print_function
from gensim.models import Word2Vec
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Merge, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.cross_validation import train_test_split
import numpy as np
import os


/Users/shimei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/shimei/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
WORD2VEC_EMBED_SIZE = 300
QA_EMBED_SIZE = 64
BATCH_SIZE = 32
NBR_EPOCHS = 40
modelname = "/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/mymodel.bin.gz"
model = Word2Vec.load(modelname)
print("Loading Word2Vec model and generating embedding matrix...")
print(model.wv['computer'])

print("Loading and formatting data...")

def get_question_answer_pairs(question_file, is_test=False):
    qapairs = []
    fqa = open(question_file, "rb")
    for line in fqa:
        if line.startswith("#"):
            continue
        line = line.strip().decode("utf8").encode("ascii", "ignore")
        cols = line.split("\t")
        question = cols[1]
        qwords = nltk.word_tokenize(question)
        if not is_test:
            correct_ans = cols[2]
            answers = cols[3:]
            # training file parsing
            correct_ans_idx = ord(correct_ans) - ord('A')
            for idx, answer in enumerate(answers):
                awords = nltk.word_tokenize(answer)
                qapairs.append((qwords, awords, idx == correct_ans_idx))
        else:
            # test file parsing (no correct answer)
            answers = cols[2:]
            for answer in answers:
                awords = nltk.word_tokenize(answer)
                qapairs.append((qwords, awords, None))
    fqa.close()
    return qapairs

qapairs = kaggle.get_question_answer_pairs(
    os.path.join(DATA_DIR, QA_TRAIN_FILE))
question_maxlen = max([len(qapair[0]) for qapair in qapairs])
answer_maxlen = max([len(qapair[1]) for qapair in qapairs])
seq_maxlen = max([question_maxlen, answer_maxlen])